In [30]:
from init import *
import time
logging.basicConfig (level=logging.INFO)

In [31]:
import ivol
from math import nan
Call = 0
Put = 1

In [32]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 100000
#model.RunTimeoutSeconds = 1

F0     = 130    # starting value of the F-process
sigma0 = 0.02  # startting value of: volatility
alpha  = 0    # Stichastic alpha (or volatility of volatility)
rho    = 0    # correlation
beta   = 0   # 
shift  = 0      # For negative rates
expiry = 2

ref_S = model.GetNumberOfStates()
model.Add(sdk.IndependentGaussian())
model.Add(sdk.CorrelatedGaussian(rho,ref_S,ref_S+1))

SABR_S_process = model.Add(sdk.Updater(
    title = 'StochVol',
    name = 'SABR_S',
    start = sigma0,
    args = [alpha]
))

assert SABR_S_process.GetStateNumber()==ref_S
assert model.GetNumberOfStates()==(SABR_S_process.GetStateNumber()+1)

SABR_F_process = model.Add(sdk.Updater(
    title = 'SABR-F',
    name = 'SABR_F',
    start = F0,
    args = [beta,shift],
    refs = [SABR_S_process.GetStateNumber()]
))
assert model.GetNumberOfStates()==(SABR_F_process.GetStateNumber()+1)

strike = 140
putOption_process = sdk.Option(SABR_F_process.GetStateNumber(),strike,sdk.Option.Put)
model.Add(putOption_process)

callOption_process = sdk.Option(SABR_F_process.GetStateNumber(),strike,sdk.Option.Call)
model.Add(callOption_process)

model.evaluations.append(sdk.EvaluationPoint(state=0,time=expiry))

print(model.json())

{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "CorrelatedGaussian", "refs": [0, 1], "args": [0]}, {"name": "SABR_S", "refs": [], "args": [0], "start": 0.02}, {"name": "SABR_F", "refs": [0], "args": [0, 0], "start": 130}, {"name": "Option", "refs": [1], "args": [140, 1], "start": 0}, {"name": "Option", "refs": [1], "args": [140, 0], "start": 0}], "evaluations": [{"state": 0, "time": 2}]}


In [33]:
result = requests.post(f'{server}/model',model.json())
er = sdk.EvaluationResults(result.json(),model)

In [34]:
er.df()

""


In [41]:
evaluation_point = 0
price_stat = er.GetStateEvaluationResult(putOption_process.GetStateNumber(),evaluation_point)
print(price_stat)
price = price_stat.mean
print(price,expiry,strike,F0)

ValueError: 

# Black Scholes pricing formula and Implied Volatility

In [42]:
spot       = 130
strike     = 110
volatility = 0.3
expiry     = 5
call_price,put_price = ivol.bs76_call_put (spot,strike,volatility,expiry)
call_price,put_price

(42.45589729822068, 22.455897298220677)

In [43]:
bs76_ivol = ivol.bs76_ivol

In [44]:
vol_call = bs76_ivol(spot,strike,expiry,call_price,'call')
vol_put  = bs76_ivol(spot,strike,expiry,put_price,'put')
vol_call, vol_put

(0.2999999944748793, 0.29999999447487924)

# Function to run SABR MonterCarlo and get back the implied volatility

In [45]:
def SABR_to_ivol (
    alpha,              # Stichastic alpha (or volatility of volatility)
    beta,               # 
    rho,                # correlation
    strike,             # 
    F0,                 # starting value of the F-process
    sigma0,             # startting value of: volatility
    expiry,             # time > 0
    shift      = 0,      # For negative rates
    TimeSteps  = 1000,
    NumPaths   = 100000,
    RandomSeed = -1,
    vol_guess  = 0.1
):
    model = sdk.Model()
    model.TimeStart = 0
    model.TimeSteps = TimeSteps
    model.NumPaths = NumPaths
    model.RunTimeoutSeconds = max(TimeSteps*NumPaths*1e-8,1)
    model.RandomSeed = RandomSeed

    ref_S = model.NumStatefulProcesses()
    model.Add(sdk.IndependentGaussian())
    model.Add(sdk.CorrelatedGaussian(rho,ref_S,ref_S+1))
    
    SABR_S_process = sdk.Updater(
        _title = 'StochVol',
        name = 'SABR_S',
        start = sigma0,
        args = [alpha]
    )
    model.Add(SABR_S_process)

    assert SABR_S_process.Number()==ref_S
    assert model.NumStatefulProcesses()==(SABR_S_process.Number()+1)

    SABR_F_process = sdk.Updater(
        _title = 'SABR-F',
        name = 'SABR_F',
        start = F0,
        args = [beta,shift],
        refs = [SABR_S_process.Number()]
    )
    model.Add(SABR_F_process)
    assert model.NumStatefulProcesses()==(SABR_F_process.Number()+1)

    putOption_process = sdk.Option(SABR_F_process.Number(),strike,sdk.Option.Put)
    model.Add(putOption_process)

    callOption_process = sdk.Option(SABR_F_process.Number(),strike,sdk.Option.Call)
    model.Add(callOption_process)

    model.AddEvaluationRequest (expiry)
    result = requests.post(f'{server}/model',model.json())
    er = sdk.EvaluationResults(result.json(),model)
    price = {}
    price['call'] = er.GetStateEvaluationResult(callOption_process.Number(),evaluation_point)
    price['put']  = er.GetStateEvaluationResult( putOption_process.Number(),evaluation_point)
    
    logging.debug(f'price_call={price["call"]}')
    logging.debug(f'price_put ={price["put"]}')

    # Take highest price
    call_put = 'call' if price['call'].MeanError() < price['put'].MeanError() else 'put'
    call_put = 'call' # FIXME: always use 'call'
    
    # if price[call_put].mean==0:
    #     return (math.nan,math.nan)
    
    val = price[call_put].mean
    err = price[call_put].MeanError()
    dx = val*1e-2 # 1% change
    logging.debug(f'{call_put} price={val} dx={dx}')
    try:
        y = bs76_ivol(F0,strike,expiry,val,call_put,vol_guess=vol_guess)
        y_dx = bs76_ivol(F0,strike,expiry,val+dx,call_put,vol_guess=vol_guess)
        deriv = (y_dx-y)/dx
        y_error = deriv*err
    except:
        y_error = math.nan
    # return (y, y_error)
    return {'call_put':call_put,'price':{'call':price['call'],'put':price['put']},'er':er,'ivol':{'value':y,'error':y_error}}

# Theory
https://en.wikipedia.org/wiki/SABR_volatility_model

In [46]:
from math import log,sqrt,pow

def SABR_theory_ivol(alpha,beta,rho,K,F0,sigma0,expiry):
    a = alpha
    b = beta
    r = rho
    f = F0
    s = sigma0
    t = expiry

    z = a/s * pow(f*K,(1-b)/2) * log(f/K)

    x = log((sqrt(1-2*r*z+z**2) + z -r)/(1-r))

    fK = pow(f*K,(1-b)/2)
    
    e1 = (1 + (pow((1-b)*s/fK,2)/24 + r*b*a*s/(4*fK) + (2-3*r**2)*a**2/24)*t)

    fKb = pow((1-b)*log(f/K),2)
    e2 = fK*(1 + fKb/24 + fKb**2/1920)

    return s*z/x * e1/e2


# Plot

In [47]:
alpha  = 0.05   # Stochastic alpha (or volatility of volatility)
beta   = 0.98   # 
rho    = 0      # correlation
F0     = 100    # starting value of the F-process
K      = 98     # strike
sigma0 = 0.05   # startting value of: volatility
expiry = 10

print(SABR_theory_ivol(alpha,beta,rho,K,F0,sigma0,expiry))
print(SABR_to_ivol (alpha,beta,rho,K,F0,sigma0,expiry,NumPaths=4))

0.04570852715627643


NameError: name 'Model' is not defined

In [ ]:
from mcsdk import linspace
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def compare_SABR (
    alpha     = 0.2,    # Stichastic alpha (or volatility of volatility)
    beta      = 0.95,   # 
    rho       = 0,      # correlation
    F0        = 100,    # starting value of the F-process
    sigma0    = 0.2,    # startting value of: volatility
    expiry    = 2,      #
    NumPaths  = 10000,
    TimeSteps = 100,
    points    = 20,     # number of points on the plot
    K         = None    # strike range
):
    if not K:
        K = (F0/2,F0*2)
    vx = linspace(K[0],K[1],points)
    vy = [ SABR_to_ivol (alpha,beta,rho,K,F0,sigma0,expiry,NumPaths=NumPaths,TimeSteps=TimeSteps)
           for K in vx ]

    # print('vy:',vy)
    # print('vy-ivol:',[v['ivol'] for v in vy])
    
    space=' \\, '
    fig = make_subplots(rows=1, cols=1)
    fig.layout.title.text = f'$\\alpha={alpha}{space}\\beta={beta}{space}\\rho={rho}{space}F_0={F0}{space}K={K}{space}\\sigma_0={sigma0}{space}T={expiry}{space}\\textrm{{paths}}={NumPaths}{space}\\textrm{{steps}}={TimeSteps}$'
    fig.append_trace(go.Scatter(
        name = 'MonteCarlo',
        x = vx,
        y = [v['ivol']['value'] for v in vy],
        error_y = {
            'type'   : 'data',
            'array'  : [v['ivol']['error'] for v in vy],
            'visible': True
        }
    ),row=1,col=1)
    fig.append_trace(go.Scatter(
        name = 'theory',
        x = vx,
        y = [
                SABR_theory_ivol (alpha,beta,rho,x,F0,sigma0,expiry)
                for x in vx
            ]
    ),row=1,col=1)
    fig.show()

In [ ]:
compare_SABR()

In [ ]:
compare_SABR(expiry=10)

In [ ]:
compare_SABR(expiry=5,rho=0.6,TimeSteps=1000,NumPaths=100000)

In [ ]:
compare_SABR(rho=-0.5)

In [ ]:
# compare_SABR(rho=0.5,TimeSteps=100)

In [ ]:
# compare_SABR(rho=0,beta=0.29,TimeSteps=10000,NumPaths=1000)

In [ ]:
# compare_SABR(alpha=0.1,rho=0,NumPaths=100000,TimeSteps=1000)

In [ ]:
# compare_SABR(alpha=0.1,rho=0,NumPaths=100000,TimeSteps=1000,K=(10,410),points=100)

## As a function of $\rho$

In [ ]:
kargs = dict(
    a = 0.2,
    b = 0.95,
    r = 0,
    f = 1.40,
    K = 1.50,
    s = 0.3,
    t = 10
)
def f(r):
    return SABR_theory_ivol(kargs['a'],kargs['b'],r,kargs['K'],kargs['f'],kargs['s'],kargs['t'])

vx = linspace(-0.99,0.99,20)
# vy = [ f(r)
#        for r in vx ]

fig = make_subplots(rows=1, cols=1)
fig.append_trace(go.Scatter(
    name = 'theory',
    x = vx,
    y = [ f(r)
          for r in vx ]
),row=1,col=1)

vy = [ SABR_to_ivol (kargs['a'],kargs['b'],r,kargs['K'],kargs['f'],kargs['s'],kargs['t'],NumPaths=100000,TimeSteps=1000,vol_guess=kargs['s'])
       for r in vx ]
fig.append_trace(go.Scatter(
    name = 'MonteCarlo',
    x = vx,
    y = [v['ivol']['value'] for v in vy],
    error_y = {
        'type'   : 'data',
        'array'  : [v['ivol']['error'] for v in vy],
        'visible': True
    }
),row=1,col=1)

fig.show()